# Benchmarking

Notebook to organise benchmarks of different implementations of the algorithms used in the `core.cnn` and `sklearn.cluster` modules.
The `CNN` clustering class exposes and wraps only one of the variants of the *fit* and the *predict* algorithm per data situation.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pre-requirements" data-toc-modified-id="Pre-requirements-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-requirements</a></span></li><li><span><a href="#Version-info" data-toc-modified-id="Version-info-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Version info</a></span></li><li><span><a href="#Helper-function-definitions" data-toc-modified-id="Helper-function-definitions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper function definitions</a></span><ul class="toc-item"><li><span><a href="#Plots" data-toc-modified-id="Plots-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Plots</a></span></li><li><span><a href="#Test-data-set-generation" data-toc-modified-id="Test-data-set-generation-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Test data set generation</a></span></li><li><span><a href="#Benchmark-organisation" data-toc-modified-id="Benchmark-organisation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Benchmark organisation</a></span></li></ul></li><li><span><a href="#Consitency-check" data-toc-modified-id="Consitency-check-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Consitency check</a></span><ul class="toc-item"><li><span><a href="#Scikit-learn-DBSCAN" data-toc-modified-id="Scikit-learn-DBSCAN-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Scikit-learn DBSCAN</a></span></li><li><span><a href="#Scikit-learn-CNN" data-toc-modified-id="Scikit-learn-CNN-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Scikit-learn CNN</a></span></li><li><span><a href="#core.cnn-CNN" data-toc-modified-id="core.cnn-CNN-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>core.cnn CNN</a></span></li></ul></li><li><span><a href="#Fit-variants" data-toc-modified-id="Fit-variants-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fit variants</a></span><ul class="toc-item"><li><span><a href="#From-neighbours" data-toc-modified-id="From-neighbours-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>From neighbours</a></span><ul class="toc-item"><li><span><a href="#From-list-of-sets" data-toc-modified-id="From-list-of-sets-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>From list of sets</a></span><ul class="toc-item"><li><span><a href="#Baseline" data-toc-modified-id="Baseline-5.1.1.1"><span class="toc-item-num">5.1.1.1&nbsp;&nbsp;</span>Baseline</a></span></li><li><span><a href="#Stdlib-index" data-toc-modified-id="Stdlib-index-5.1.1.2"><span class="toc-item-num">5.1.1.2&nbsp;&nbsp;</span>Stdlib index</a></span></li><li><span><a href="#Stdlib-cython" data-toc-modified-id="Stdlib-cython-5.1.1.3"><span class="toc-item-num">5.1.1.3&nbsp;&nbsp;</span>Stdlib cython</a></span></li></ul></li><li><span><a href="#From-numpy.array" data-toc-modified-id="From-numpy.array-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>From numpy.array</a></span></li><li><span><a href="#Check-in-CNN-class-context" data-toc-modified-id="Check-in-CNN-class-context-5.1.3"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Check in CNN class context</a></span></li></ul></li></ul></li></ul></div>

## Pre-requirements

In [13]:
from collections import Counter
import importlib
from operator import itemgetter
import sys
import time


from IPython.core.display import display, HTML
import numpy as np
%matplotlib widget
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import cluster
from sklearn import datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

from core import _cfits             # Cythonised fit implementation snippets
from core import cnn
from snippets import fits  # Fit mplementation snippets

# Jupyter extensions
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [14]:
# Matplotlib configuration
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rc_file("../../tutorial/matplotlibrc", use_default_template=False)

In [15]:
# Jupyter notebook configuration
display(HTML("<style>.container { width:85% !important; }</style>"))

## Version info

In [16]:
print(sys.version)

3.8.1 (default, Jan 23 2020, 07:59:15) 
[GCC 8.3.0]


## Helper function definitions

### Plots

In [17]:
# Axis property defaults for the plots
ax_props = {
    "xlabel": None,
    "ylabel": None,
    "xlim": (-2.5, 2.5),
    "ylim": (-2.5, 2.5),
    "xticks": (),
    "yticks": (),
    "aspect": "equal"
}

In [18]:
def plot_data(data, labels=None, ax=None, noise=0):
    """Take a data set and cluster labels to make a basic 2D dot plot
    
    Args:
        data: Numpy `numpy.ndarray` of shape (#points, 2) with x, y
            coordinates of points in 2D
        labels: Numpy `numpy.ndarray` of shape (#points,) and
            `dtype = int` holding cluster label assignments for all
            points.  If `None`, will plot the data set without
            point colouring by label.
        ax: Matplotlib `matplotlib.axes.SubplotBase` instance to
            attach the plot to.  If `None`, wil create a new instance.
        noise: Integer label used to mark point as noise (no cluster
            assignment; Usually 0 or -1). 
    """

    if ax is None:
        plt.close('all')
        fig, ax = plt.subplots(
            figsize=(
                mpl.rcParams["figure.figsize"][0] / 2,
                mpl.rcParams["figure.figsize"][1]
                )
            )
    else:
        fig = ax.get_figure()
        
    if labels is None:
        ax.plot(
            *data.T,
            linestyle="",
            color="None",
            marker="o",
            markersize=4,
            markerfacecolor="white",
            markeredgecolor="k",
            )

    else:
        ax.plot(
            *data[np.where(labels == noise)[0]].T,
            linestyle="",
            color="None",
            marker="o",
            markersize=4,
            markerfacecolor="gray",
            markeredgecolor="k",
            )

        for cluster_number in range(noise + 1 , int(np.max(labels)) + 1):
            ax.plot(
                *data[np.where(labels == cluster_number)[0]].T,
                linestyle="",
                marker="o",
                markersize=4,
                markeredgecolor="k",
                )

    ax.set(**{
        "xticks": (),
        "yticks": (),
        "xticklabels": (),
        "yticklabels": (),
        "aspect": "equal"
        })

### Test data set generation

Data set generation function should be generally designed in such a way that the expect exactly one argument *n* and return a 2D data set of *n* sample points. A `label` attribute can be optionally added to the function object for identification.

In [19]:
# Global seed for data set generation functions
np.random.seed(42)

In [20]:
# circles
def gen_circles(n):
    circles, _ = datasets.make_circles(
        n_samples=n,
        factor=.5,
        noise=.05,
        random_state=10
        )
    
    return StandardScaler().fit_transform(circles)

gen_circles.label = "circles"

# blobs                            
def gen_blobs(n):
    blobs, _ = datasets.make_blobs(
        centers=[[-10, -10], [10, -10], [10, 10]],
        n_samples=n,
        random_state=10
    )
    return StandardScaler().fit_transform(blobs)

gen_blobs.label = "blobs"

# moons
def gen_moons(n):
    moons, _ = datasets.make_moons(
        n_samples=n,
        noise=.05,
        random_state=10
        )
    
    return StandardScaler().fit_transform(moons)

gen_moons.label = "moons"

def gen_no_structure(n):
    no_structure = np.random.rand(n, 2)
    
    return StandardScaler().fit_transform(no_structure)

gen_no_structure.label = "None"

def gen_aniso(n):
    X, y = datasets.make_blobs(
        n_samples=n,
        random_state=170
        )

    transformation = [[0.6, -0.6], [-0.4, 0.8]]
    aniso = np.dot(X, transformation)
    
    return StandardScaler().fit_transform(aniso)

gen_aniso.label = "aniso"

def gen_varied(n):
    varied, _ = datasets.make_blobs(
        n_samples=n,
        cluster_std=[1.0, 2.5, 0.5],
        random_state=170)
    
    return StandardScaler().fit_transform(varied)

gen_varied.label = "varied"

### Benchmark organisation

In [21]:
class DS:
    """Benchmark class to represent a data set
    
    Initialise a `DS` instance with a data set generation function.
    
    Attributes:
       data: A (2D) sample data set
       prepare_neighbourhoods: Pre-compute neighbourhoods
    """

    def __init__(
            self, gen_fxn, n, gen_fxn_args=None, gen_fxn_kwargs=None):
        self.gen_fxn = gen_fxn
        self.n = n
        
        if gen_fxn_kwargs is None:
            gen_fxn_kwargs = {}
        if gen_fxn_args is None:
            gen_fxn_args = ()  
        
        self.data = gen_fxn(n, *gen_fxn_args, **gen_fxn_kwargs)
        self.neighbourhoods = None  # Pre-computed neighbourhoods 
        self.r = None               # Neighbourhood computation radius
        self.timeits = {}           # Timing results
        
    def prepare_neighbourhoods(self, r, numpy=False):
        """Pre-compute neighbourhoods at a given readius `r`
        
        Uses `sklearn.neighbors.NearestNeighbors`.
        
        Args:
            numpy: If `True`, returns a `numpy.ndarray` of shape
            (#points,) containing the neighbours of each points as
            `numpy.ndarray` of shape (#neighbours,).  If `False`,
            returns a `list` of `sets` instead.
        """

        neighbour_model = NearestNeighbors(radius=r).fit(self.data)
        neighbours = neighbour_model.radius_neighbors(
            self.data, return_distance=False
            )
        
        # Remove self-counting
        neighbours = [set(x) for x in neighbours]   
        for c, s in enumerate(neighbours):
            s.remove(c)
        if numpy:
            neighbours = np.array([np.array([y for y in x]) for x in neighbours])    
        
        self.neighbourhoods = neighbours
        self.r = r
        
    def __str__(self):
        try:
            desc_gen = self.gen_fxn.label
        except AttributeError:
            desc_gen = self.gen_fxn.__name__
            
        # p: from points
        # d: from distances
        # n: from neighbours
        
        if self.neighbourhoods is not None:
            desc_from = f"n{self.r}"
        elif self.dist is not None:
            desc_from = f"d"
        else:
            desc_from = f"p"
            
        return f"{desc_gen}_{self.n}_{desc_from}"
    
    
    def ratios(self, base=None):
        """Show relative performance of runs based on `timeits` dict"""

        if base is not None:
            base = self.timeits[base].average
        else:
            base = min(x.average for x in self.timeits.values())
            
        return sorted([
            (k, v.average / base)
            for k, v in self.timeits.items()
            ], key=itemgetter(1))
        

In [22]:
def profile_fxn(
        f, ds, report_dir, *args,
        t=True, l=True, m=True, label=None, **kwargs):
    """Function profiling procedure
    
    Runs %lprun and %timeit line magic on a globally defined function
    `fxn`.  Function args and kwargs need to be defined globally as
    well.  This is necessary, because (at least lprun) line magic does
    not seem to work well with local variables, e.g. the following
    alternative did not work (raises `NameError`): 
    
       def profile_fxn(fxn, *args, **kwargs):
           %lprun -f fxn fxn(*args, **kwargs)
           ...
    
    This function expects a :obj:`DS` object, providing a dataset and
    pre-calculated values if necessary.  Report details are deduced from
    this object.  Timings are saved to the object.
    
    Args:
       f (:obj:`func`): Function to profile.
       ds (:obj:`DS`): Data set object.
       report_dir (str): Output directory file path.
       *args: Arguments passed to `f`
    
    Keyword args:
       label (optional, str): Label to identify the run.  If `None`,
          `fxn.__name__` is used.
       *kwargs: Keyword arguments passed to `f`.
          
    
    Returns:
       None
    
    """
    
    global fxn
    global fxn_args
    global fxn_kwargs
    
    fxn = f
    fxn_args = args
    fxn_kwargs = kwargs
    
    # Validate function result
    result = fxn(*fxn_args, **fxn_kwargs)
    
    # Convert result if not labels array
    if isinstance(result, list) and isinstance(result[0], np.ndarray):
        # Convert result if from original implementation (baseline)
        result = baseline_to_labels(result)
    
    if result is not None:
        noise = 0
        frequencies = Counter(result)
        if 0 in frequencies:
            noise = frequencies.pop(0)

        largest = frequencies.most_common(1)[0][1] if frequencies else 0
        clusters = len(frequencies)

        print(f"Length of labels:    {len(result)}")
        print(f"Noise:               {noise}")
        print(f"Largest:             {largest}")
        print(f"Clusters:            {clusters}")

    
    # Profile
    if l:
        %lprun -T {report_dir}/{fxn.__name__}_{ds.__str__()}.lprun -f fxn fxn(*fxn_args, **fxn_kwargs)
    if t:
        o = %timeit -o fxn(*fxn_args, **fxn_kwargs)

        if label is None:
            label = fxn.__name__
        ds.timeits.update({label: o})

In [23]:
def baseline_to_labels(result):
    """Convert result from original implementation (baseline)"""

    len_ = len(result)
    result = [x for x in result if isinstance(x, np.ndarray)]
    result_ = np.zeros(len_)
    for c, cluster in enumerate(result, 1):
        for member in cluster:
            result_[member] = c
    result = result_

    return result

## Consitency check

Ensure that every implementation delivers a consistent cluster result for test data sets.

In [18]:
# Plot the test data sets
plt.close("all")
fig, ax = plt.subplots(2, 3)
Ax = ax.flatten()

generation_fxns = [
    gen_circles, gen_moons, gen_varied,
    gen_aniso, gen_blobs, gen_no_structure,
    ]

for count, gen_fxn in enumerate(generation_fxns):
    # Plot
    plot_data(gen_fxn(5000), ax=Ax[count])
    Ax[count].set(**ax_props)
    
    try:
        name = gen_fxn.label
    except AttributeError:
        name = gen_fxn.__name__

    Ax[count].set_title(f'{name}', fontsize=10, pad=4)
    
fig.subplots_adjust(
    left=0, right=1, bottom=0.05, top=0.9, wspace=0, hspace=0.2 
    )

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Scikit-learn DBSCAN

In [19]:
# Plot the test data sets
plt.close("all")
fig, ax = plt.subplots(2, 3)
Ax = ax.flatten()

generation_fxns = [
    gen_circles, gen_moons, gen_varied,
    gen_aniso, gen_blobs, gen_no_structure,
    ]

fit_params = [
    {"eps": 0.2, "min_samples": 5},    # circles
    {"eps": 0.2, "min_samples": 5},    # moons
    {"eps": 0.14, "min_samples": 20},  # varied
    {"eps": 0.11, "min_samples": 20},  # aniso
    {"eps": 0.2, "min_samples": 5},    # blobs
    {"eps": 0.2, "min_samples": 5},    # no structure
    ]

for count, (gen_fxn, params) in enumerate(zip(generation_fxns, fit_params)):
    # Fit
    data = gen_fxn(5000)
    labels = cluster.dbscan(data, **params)[1]
    
    # Plot
    plot_data(data, labels=labels, ax=Ax[count], noise=-1)
    Ax[count].set(**ax_props)
    
    try:
        name = gen_fxn.label
    except AttributeError:
        name = gen_fxn.__name__

    Ax[count].set_title(f'{name}', fontsize=10, pad=4)
    
fig.subplots_adjust(
    left=0, right=1, bottom=0.05, top=0.9, wspace=0, hspace=0.2 
    )

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Scikit-learn CNN

In [20]:
# Plot the test data sets
plt.close("all")
fig, ax = plt.subplots(2, 3)
Ax = ax.flatten()

generation_fxns = [
    gen_circles, gen_moons, gen_varied,
    gen_aniso, gen_blobs, gen_no_structure,
    ]

fit_params = [
    {"eps": 0.2, "min_samples": 5},    # circles
    {"eps": 0.2, "min_samples": 5},    # moons
    {"eps": 0.18, "min_samples": 20},  # varied
    {"eps": 0.15, "min_samples": 10},  # aniso
    {"eps": 0.2, "min_samples": 5},    # blobs
    {"eps": 0.2, "min_samples": 5},    # no structure
    ]

for count, (gen_fxn, params) in enumerate(zip(generation_fxns, fit_params)):
    # Fit
    data = gen_fxn(5000)
    labels = cluster.cnn(data, **params)
    
    # Plot
    plot_data(data, labels=labels, ax=Ax[count], noise=-1)
    Ax[count].set(**ax_props)
    
    try:
        name = gen_fxn.label
    except AttributeError:
        name = gen_fxn.__name__

    Ax[count].set_title(f'{name}', fontsize=10, pad=4)
    
fig.subplots_adjust(
    left=0, right=1, bottom=0.05, top=0.9, wspace=0, hspace=0.2 
    )

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
# Plot the test data sets
plt.close("all")
fig, ax = plt.subplots(2, 3)
Ax = ax.flatten()

generation_fxns = [
    gen_circles, gen_moons, gen_varied,
    gen_aniso, gen_blobs, gen_no_structure,
    ]

fit_params = [
    {"radius_cutoff": 0.2, "cnn_cutoff": 5},                        # circles
    {"radius_cutoff": 0.2, "cnn_cutoff": 5},                        # moons
    {"radius_cutoff": 0.18, "cnn_cutoff": 20, "member_cutoff": 8},  # varied
    {"radius_cutoff": 0.15, "cnn_cutoff": 10},                      # aniso
    {"radius_cutoff": 0.2, "cnn_cutoff": 5},                        # blobs
    {"radius_cutoff": 0.2, "cnn_cutoff": 5},                        # no structure
    ]

for count, (gen_fxn, params) in enumerate(zip(generation_fxns, fit_params)):
    # Fit
    data = gen_fxn(5000)
    clustering = cnn.CNN(data)
    clustering.calc_dist()
    clustering.fit(**params, rec=False)
    
    # Plot
    plot_data(data, labels=clustering.labels, ax=Ax[count], noise=0)
    Ax[count].set(**ax_props)
    
    try:
        name = gen_fxn.label
    except AttributeError:
        name = gen_fxn.__name__

    Ax[count].set_title(f'{name}', fontsize=10, pad=4)
    
fig.subplots_adjust(
    left=0, right=1, bottom=0.05, top=0.9, wspace=0, hspace=0.2 
    )

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### core.cnn CNN

## Fit variants

Benchmarks for different approaches to the CNN fit function.

In [42]:
importlib.reload(fits)
importlib.reload(cfits)

<module 'cfits' from '/home/janjoswig/CNN/cfits.cpython-38-x86_64-linux-gnu.so'>

In [26]:
# Benchmark results will be saved under:
report_dir = "reports/T460"
# report_dir = "reports/qcw21"
# report_dir = "reports/qcm07"

### From neighbours

Tests of fit functions taking pre-computed neighbourhoods as input. Two ways of setting up the benchmarks are provided:
  - Use `prepare_neighbours` function to quickly generate the input data
  - Use `DS` benchmark class to organise different runs on essentially the same data set

In [13]:
# Benchmark fixture 1
def prepare_neighbours(n, r, gen_fxn, numpy=False):
    """Provide pre-computed neighbourhoods
    
    Uses `sklearn.neighbors.NearestNeighbors`. Removes self-counting
    of points as neighbours of themselves.
    
    Args:
       n: Number of data points
       r: Radius
       gen_fxn: Function that accepts one parameter `n` and returns
           a data set with `n` points, for which neighbourhoods will
           be computed.
       numpy: If `True`, provide neighbourhoods as 1D `numpy.array` of 1D
           `numpy.array`s.  If `False`, convert to `list` of `set`s.  
    """
    
    data = gen_fxn(n)
    neighbour_model = NearestNeighbors(radius=r).fit(data)
    neighbours = neighbour_model.radius_neighbors(data, return_distance=False)
    neighbours = [set(x) for x in neighbours]
    for c, s in enumerate(neighbours):
        # Remove self-counting 
        s.remove(c)
    if numpy:
        # Reconvert to numpy.array
        neighbours = np.array([np.array([y for y in x]) for x in neighbours])
        
    return neighbours

In [78]:
# 10000 points of circles data set
n = 5000
r = 0.25
gen_fxn = gen_circles
ds = gen_fxn.label
neighbours = prepare_neighbours(n, r, gen_fxn)

In [27]:
# 100 points from circles data set
ds = DS(gen_circles, 2000)

#### From list of sets

In [28]:
# Prepare neighbours as list of sets
ds.prepare_neighbourhoods(0.5)
print(ds)

circles_2000_n0.5


##### Baseline

In [35]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=baseline_to_labels(fits.fit_from_neighbours_baseline(20, ds.neighbours)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
# Implementation using original implementation
profile_fxn(
    fits.fit_from_neighbours_baseline,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="baseline"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_from_neighbours_baseline_circles_2000_n0.5.lprun'. 
256 ms ± 5.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


##### Stdlib index

In [47]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=np.asarray(fits.fit_stdlib_from_neighbours_index(20, ds.neighbourhoods)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
# Implementation using only standard library
profile_fxn(
    fits.fit_stdlib_from_neighbours_index,
    ds, report_dir,
    20, ds.neighbourhoods,  # function args
    label="std_index"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_stdlib_from_neighbours_index_circles_2000_n0.5.lprun'. 
36.3 ms ± 88 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [111]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=np.asarray(fits.fit_stdlib_from_neighbours_loop(20, ds.neighbours)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [113]:
# Implementation using only standard library
profile_fxn(
    fits.fit_stdlib_from_neighbours_loop,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="std_loop"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_stdlib_from_neighbours_loop_circles_2000_n0.5.lprun'. 
40 ms ± 511 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [114]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=np.asarray(fits.fit_stdlib_from_neighbours_loop_membercheck(20, ds.neighbours)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [115]:
# Implementation using only standard library
profile_fxn(
    fits.fit_stdlib_from_neighbours_loop_membercheck,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="std_loop_membercheck"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_stdlib_from_neighbours_loop_membercheck_circles_2000_n0.5.lprun'. 
41.8 ms ± 3.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### Stdlib cython

In [48]:
from core import _cfits

In [29]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=np.asarray(_cfits.fit_from_neighbours(20, ds.neighbourhoods)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
# Implementation using only standard library
profile_fxn(
    _cfits.fit_from_neighbours,
    ds, report_dir,
    20, ds.neighbourhoods,  # function args
    label="std_cython"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2


ValueError: max() arg is an empty sequence

In [32]:
%timeit _cfits.fit_from_neighbours(20, ds.neighbourhoods)

25.8 ms ± 541 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### From numpy.array

In [136]:
importlib.reload(fits)

<module 'snippets.fits' from '/home/janjoswig/CNN/tests/benchmark/snippets/fits.py'>

In [118]:
# Switch to neighbourhoods as numpy.array of numpy.arrays
ds.prepare_neighbours(0.5, numpy=True)

In [142]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=baseline_to_labels(fits.fit_from_neighbours_baseline(20, ds.neighbours)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [140]:
# Implementation using original implementation
profile_fxn(
    fits.fit_from_neighbours_baseline,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="baseline"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_from_neighbours_baseline_circles_2000_n0.5.lprun'. 
368 ms ± 57.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [137]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=fits.fit_numpy_mix(20, ds.neighbours), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
# Implementation using numpy
profile_fxn(
    fits.fit_numpy_mix,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="numpy_index"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_numpy_mix_circles_2000_n0.5.lprun'. 
146 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [119]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=np.asarray(fits.fit_numpy_from_neighbours_index(20, ds.neighbours)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [139]:
# Implementation using numpy
profile_fxn(
    fits.fit_numpy_from_neighbours_index,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="numpy_index"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_numpy_from_neighbours_index_circles_2000_n0.5.lprun'. 
103 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [121]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=np.asarray(fits.fit_numpy_from_neighbours_loop(20, ds.neighbours)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [123]:
# Implementation using numpy
profile_fxn(
    fits.fit_numpy_from_neighbours_loop,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="numpy_loop"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_numpy_from_neighbours_loop_circles_2000_n0.5.lprun'. 
96.9 ms ± 2.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [124]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=np.asarray(fits.fit_numpy_from_neighbours_filtermembers(20, ds.neighbours)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
# Implementation using numpy
profile_fxn(
    fits.fit_numpy_from_neighbours_filtermembers,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="numpy_filter"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_numpy_from_neighbours_filtermembers_circles_2000_n0.5.lprun'. 
43.8 ms ± 965 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [132]:
importlib.reload(fits)

<module 'snippets.fits' from '/home/janjoswig/CNN/tests/benchmark/snippets/fits.py'>

In [129]:
plt.close("all")
fig, ax = plt.subplots()
plot_data(ds.data, labels=np.asarray(fits.fit_numpy_from_neighbours_membercheck(20, ds.neighbours)), ax=ax, noise=0)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [131]:
# Implementation using numpy
profile_fxn(
    fits.fit_numpy_from_neighbours_membercheck,
    ds, report_dir,
    20, ds.neighbours,  # function args
    label="numpy_filter"
    )

Length of labels:    2000
Noise:               0
Largest:             1000
Clusters:            2

*** Profile printout saved to text file 'reports/T460/fit_numpy_from_neighbours_membercheck_circles_2000_n0.5.lprun'. 
46.7 ms ± 4.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [73]:
# Implementation using cythonised numpy
profile_fxn(
    cfits.cfit_from_neighbours,
    ds, report_dir,
    1, ds.neighbours,  # function args
    label="cython_numpy_loop",
    l=False
    )

ValueError: Buffer dtype mismatch, expected 'npy_intp' but got 'double'

In [34]:
for x in ds.ratios():
    print(f"{x[0]:>15}: {x[1]:7.3f}")

        std_cnn:   1.000


#### Check in CNN class context

In [33]:
cobj = cnn.CNN(neighbourhoods=cnn.Neighbourhoods(ds.neighbourhoods, 0.5))
profile_fxn(
    cobj.fit,
    ds, report_dir,
    0.5, 20,  # function args
    label="std_cnn",
    rec=False
    )


*** Profile printout saved to text file 'reports/T460/fit_circles_2000_n0.5.lprun'. 
29.1 ms ± 598 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
cobj = cluster.CNN(neighbourhoods=cnn.Neighbourhoods(ds.neighbourhoods, 0.5))
profile_fxn(
    cobj.fit,
    ds, report_dir,
    0.5, 20,  # function args
    label="std_cnn",
    rec=False
    )

In [27]:
importlib.reload(cfits)

<module 'cfits' from '/home/janjoswig/CNN/cfits.cpython-38-x86_64-linux-gnu.so'>

In [26]:
%%cython --annotate

# distutils: language = c++
# cython: boundcheck = False
# cython: wraparound = False

cimport cython
from libcpp.vector cimport vector

def fit_from_neighbours(
        cnn_cutoff: int,
        neighbourhoods: list):
    """Worker function variant applying the CNN algorithm.

    Assigns labels to points starting from pre-computed neighbourhoods.
    Uses Python standard library only.

    Args:
        cnn_cutoff: Points need to share at least this many neighbours
            to be assigned to the same cluster (similarity criterion).
        neighbourhoods: List of length #points containing sets of
            neighbouring point indices

    Returns:
        Labels
    """

    cdef int n, m, k
    cdef vector[int] labels
    cdef vector[int] consider
    cdef vector[int] stack
    cdef int current
    cdef set neighbours, neighbour_neighbours
    cdef int init_point, point, member_i, member

   # Number of points
    n = len(neighbourhoods)

    # Initialise labels
    labels = [0 for _ in range(n)]

    # Track assigment
    consider = [1 for _ in range(n)]

    # Start with first cluster (0 = noise)
    current = 1

    for init_point in range(n):
        # Loop over points
        if consider[init_point] == 0:
            # Point already assigned
            continue

        neighbours = neighbourhoods[init_point]
        m = len(neighbours)
        if m <= cnn_cutoff:
            # Point can not fulfill cnn condition
            labels[init_point] = 0             # Assign cluster label
            consider[init_point] = 0           # Mark point as included
            continue

        labels[init_point] = current          # Assign cluster label
        consider[init_point] = 0              # Mark point as included

        point = init_point
        while True:
            # Loop over neighbouring points
            for member_i in range(m):
                member = neighbours[member_i]
                if consider[member] == 0:
                    # Point already assigned
                    continue

                neighbour_neighbours = neighbourhoods[member]
                k = len(neighbour_neighbours)
                if k <= cnn_cutoff:
                    labels[member] = 0
                    consider[member] = 0
                    continue

                # conditional growth
                if (len(neighbours.intersection(neighbourhoods[member]))
                        >= cnn_cutoff):
                    labels[member] = current
                    consider[member] = 0
                    stack.push_back(member)

            if stack.size() == 0:
                # No points left to check
                break

            while stack.size() > 0:
                point = stack.back()  # Get the next point from the queue
                stack.pop_back()
                neighbours = neighbourhoods[point]
                m = len(neighbours)
                if m <= cnn_cutoff:
                    # Point can not fulfill cnn condition
                    labels[init_point] = 0             # Assign cluster label
                    consider[init_point] = 0           # Mark point as included
                    continue
                break

        current += 1

    return labels